##  Aggregations


### Step 1: Initialize PySpark Session


In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("day3").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/03 23:13:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2: Load the Dataset


In [2]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "../data/US_Crime_Rates_1960_2014.csv"  # Replace with the actual path
US_Crime_Rates_1960_2014_df = spark.read.csv(data_path, header=True, inferSchema=True)

# Load the Chipotle dataset into a Spark DataFrame
data_path = "../data/titanic.csv"  # Replace with the actual path
titanic_df = spark.read.csv(data_path, header=True, inferSchema=True)


In [3]:
US_Crime_Rates_1960_2014_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Violent: integer (nullable = true)
 |-- Property: integer (nullable = true)
 |-- Murder: integer (nullable = true)
 |-- Forcible_Rape: integer (nullable = true)
 |-- Robbery: integer (nullable = true)
 |-- Aggravated_assault: integer (nullable = true)
 |-- Burglary: integer (nullable = true)
 |-- Larceny_Theft: integer (nullable = true)
 |-- Vehicle_Theft: integer (nullable = true)



In [4]:
US_Crime_Rates_1960_2014_df.show()


+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964| 191141000| 4564600| 364220| 4200400|  9360|        21420| 130390|            203050| 12132

### count

Question: How many records are there in the US_Crime_Rates_1960_2014_df DataFrame?

In [38]:
ccount = US_Crime_Rates_1960_2014_df.count()

print("Number of records: ", ccount)

Number of records:  55


In [41]:
#temp view for sql 
US_Crime_Rates_1960_2014_df.createOrReplaceTempView('crime_us')

In [42]:
#sql

count = spark.sql("""
                      select count(*) as number_of_count
                      from crime_us
                     """)


print("Number of Count:", count.collect()[0][0])

Number of Count: 55


### countDistinct
Question: How many distinct years are present in the US_Crime_Rates_1960_2014_df DataFrame?
Answer:

In [43]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, countDistinct ,approxCountDistinct, first , last , max,min , sumDistinct,avg,struct,sum

In [7]:

count_year = US_Crime_Rates_1960_2014_df.select(countDistinct('Year'))
print("Number of Distinct Years: ", count_year.collect()[0][0])


Number of Distinct Years:  55


In [45]:
#sql

distinct_count = spark.sql("""
                        select count(distinct 'year') as distinct_count_of_years
                       from crime_us
                         """)   
# distinct_count.show()
print("Number of distinct years:",distinct_count.collect()[0][0])

Number of distinct years: 1


### approx_count_distinct

Question: Estimate the approximate number of distinct values in the "Total" column of the US_Crime_Rates_1960_2014_df DataFrame.

In [8]:
#here we use approxCountDistinct with max estimation error of 0.1
US_Crime_Rates_1960_2014_df.select(approxCountDistinct('Total',0.1)).show()
print("",)


/opt/spark/python/pyspark/sql/functions.py:2610: FutureWarning: Deprecated in 2.1, use approx_count_distinct instead.
  warnings.warn("Deprecated in 2.1, use approx_count_distinct instead.", FutureWarning)


+----------------------------+
|approx_count_distinct(Total)|
+----------------------------+
|                          51|
+----------------------------+



###  first and last

Question: Find the first and last year in the US_Crime_Rates_1960_2014_df DataFrame.

In [9]:
first = US_Crime_Rates_1960_2014_df.select(first('Year'))
last  = US_Crime_Rates_1960_2014_df.select(last('Year'))
print("First Year: ", first.collect()[0][0])
print("Last Year: ", last.collect()[0][0])

First Year:  1960
Last Year:  2014


### min and max

Question: Find the minimum and maximum population values in the US_Crime_Rates_1960_2014_df DataFrame.

In [10]:
min = US_Crime_Rates_1960_2014_df.select(min("Population"))
max = US_Crime_Rates_1960_2014_df.select(max("Population"))

print("Minimum Population",min.collect()[0][0])
print("Maximum Population",max.collect()[0][0])

Minimum Population 179323175
Maximum Population 318857056


### sumDistinct

Question: Calculate the sum of distinct "Property" values for each year in the US_Crime_Rates_1960_2014_df DataFrame.

In [11]:
#in this problem , we first use the groupby clause to group the property value for each year 
# then we use the funcion sumDistinct to do so .
# we can also use the method sum_distinct mehtod because of the future deprecation of the previous function 

distinct_sum = US_Crime_Rates_1960_2014_df.groupBy('Year').agg(sumDistinct(col('Property')).alias('SumDistinctYear'))
distinct_sum.show()

/opt/spark/python/pyspark/sql/functions.py:752: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----+---------------+
|Year|SumDistinctYear|
+----+---------------+
|1990|       12655500|
|1975|       10252700|
|1977|        9955000|
|2003|       10442862|
|2007|        9843481|
|1974|        9278700|
|2006|        9983568|
|1978|       10123400|
|1961|        3198600|
|2013|        8650761|
|1988|       12356900|
|1997|       11558175|
|1994|       12131900|
|1968|        6125200|
|2014|        8277829|
|1973|        7842200|
|1979|       11041500|
|1971|        7771700|
|1966|        4793300|
|2004|       10319386|
+----+---------------+
only showing top 20 rows



### avg

Question: Calculate the average "Murder" rate for the entire dataset in the US_Crime_Rates_1960_2014_df DataFrame.
Answer:

In [12]:
avg_murder = US_Crime_Rates_1960_2014_df.select(avg(col("Murder")))
print("Average Murder Rate: ", avg_murder.collect()[0][0])

Average Murder Rate:  17317.236363636363


In [13]:
US_Crime_Rates_1960_2014_df.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964| 191141000| 4564600| 364220| 4200400|  9360|        21420| 130390|            203050| 12132

### Aggregating to Complex Types

Question: Calculate the total sum of "Violent" and "Property" crimes for each year in the US_Crime_Rates_1960_2014_df DataFrame. Store the results in a struct type column.

In [14]:
distinct_sum = US_Crime_Rates_1960_2014_df.groupBy('Year').agg(
    struct(
    sum(col('Property')),
    sum(col('Violent'))
    ).alias("CrimeSums")
)
distinct_sum.show()


+----+-------------------+
|Year|          CrimeSums|
+----+-------------------+
|1990|{12655500, 1820130}|
|1975|{10252700, 1039710}|
|1977| {9955000, 1029580}|
|2003|{10442862, 1383676}|
|2007| {9843481, 1408337}|
|1974|  {9278700, 974720}|
|2006| {9983568, 1418043}|
|1978|{10123400, 1085550}|
|1961|  {3198600, 289390}|
|2013| {8650761, 1199684}|
|1988|{12356900, 1566220}|
|1997|{11558175, 1634770}|
|1994|{12131900, 1857670}|
|1968|  {6125200, 595010}|
|2014| {8277829, 1197987}|
|1973|  {7842200, 875910}|
|1979|{11041500, 1208030}|
|1971|  {7771700, 816500}|
|1966|  {4793300, 430180}|
|2004|{10319386, 1360088}|
+----+-------------------+
only showing top 20 rows



### Grouping

Question: In the given US_Crime_Rates_1960_2014_df DataFrame, you are tasked with finding the average of all crimes combined for each year. Calculate the sum of all crime categories (Violent, Property, Murder, Forcible_Rape, Robbery, Aggravated_assault, Burglary, Larceny_Theft, Vehicle_Theft) for each year and then determine the average of these combined crime sums. Provide the result as the average of all crimes across the entire dataset.

In [15]:
# first we calculated the total sum of al the crimes in a year 
# and then we averaged that and we showed as per the expected outcome
sum_crime_year = US_Crime_Rates_1960_2014_df.withColumn(
    'TotalCrimes',
    col("Violent") + col("Property") + col("Murder") + col("Forcible_Rape") + col("Robbery") + col("Aggravated_assault") + col("Burglary") + col("Larceny_Theft") + col("Vehicle_Theft")
)


average_crime = sum_crime_year.select(avg(col("TotalCrimes")))

print("Average of all crimes: ",average_crime.collect()[0][0])

sum_crime_year.select("Year","TotalCrimes").show()

Average of all crimes:  21201546.145454545
+----+-----------+
|Year|TotalCrimes|
+----+-----------+
|1960|    6768320|
|1961|    6975980|
|1962|    7504420|
|1963|    8218940|
|1964|    9129240|
|1965|    9478780|
|1966|   10446960|
|1967|   11806860|
|1968|   13440420|
|1969|   14821740|
|1970|   16196040|
|1971|   17176400|
|1972|   16497600|
|1973|   17436220|
|1974|   20506940|
|1975|   22584730|
|1976|   22699410|
|1977|   21969060|
|1978|   22417910|
|1979|   24499060|
+----+-----------+
only showing top 20 rows



### Window Functions

Question: Calculate the cumulative sum of "Property" values over the years using a window function in the US_Crime_Rates_1960_2014_df DataFrame.

In [16]:
wind_spec = Window.orderBy("Year")
cum_sum_prop_df = US_Crime_Rates_1960_2014_df.withColumn(
    "CumulativePropertySum",
    sum(col("Property")).over(wind_spec)
)

cum_sum_prop_df.show()

23/09/03 23:13:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/03 23:13:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/03 23:13:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/03 23:13:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/03 23:13:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+---------------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|CumulativePropertySum|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+---------------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|              3095700|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|              6294300|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|              9745000|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 10

### Pivot
Question: You are working with a DataFrame named US_Crime_Rates_1960_2014_df that contains crime data for different crime types over the years. 

In [37]:
#this gives us the result as expected. 
pivot_df = US_Crime_Rates_1960_2014_df.groupBy("Year").pivot("Year").agg({"Robbery": "max"})
pivot_df.orderBy('Year').show()

+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|Year|  1960|  1961|  1962|  1963|  1964|  1965|  1966|  1967|  1968|  1969|  1970|  1971|  1972|  1973|  1974|  1975|  1976|  1977|  1978|  1979|1980|1981|1982|1983|1984|1985|1986|1987|1988|1989|1990|1991|1992|1993|1994|1995|1996|1997|1998|1999|2000|2001|2002|2003|2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|
+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|1960|107840|  null|  null|  null|